In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t, f, multivariate_normal, norm
import numpy as np
import statsmodels.api as sm
from Linear_Reg_Diagnostic import Linear_Reg_Diagnostic
import statsmodels.stats.outliers_influence as smoi

1:

In [54]:
def gen(n):
    X = multivariate_normal.rvs(mean=np.zeros(10), cov=np.identity(10), size=n)
    Y = 1 + 0.1 * X.T[0] + norm.rvs(size=n)
    return X, Y


In [119]:
X, Y = gen(100)
X = sm.add_constant(X)
model = sm.OLS(Y, X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     1.318
Date:                Thu, 20 Apr 2023   Prob (F-statistic):              0.233
Time:                        16:17:48   Log-Likelihood:                -141.85
No. Observations:                 100   AIC:                             305.7
Df Residuals:                      89   BIC:                             334.3
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9985      0.114      8.766      0.0

In [72]:
coff = np.zeros(11)
coff[0] = 1
coff[1] = 0.1

def select_and_check(interval, p_val, index):
    return p_val <  0.1 if (interval[0] < coff[index] < interval[1]) else np.NAN


In [73]:
def select_and_check_vector(results):
    intervals = results.conf_int()
    pvals = results.pvalues
    return np.array([select_and_check(intervals[i], pvals[i], i) for i in range(11)])

In [74]:
def gen_results(n):
    X, Y = gen(n)
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    return model.fit()

In [75]:
array = np.array([select_and_check_vector(gen_results(100)) for i in range(5000)])

In [130]:
array[~np.isnan(array).any(axis=1)].mean(axis=0)

array([1.        , 0.2109375 , 0.05298913, 0.05264946, 0.04381793,
       0.05774457, 0.04755435, 0.05129076, 0.05264946, 0.05095109,
       0.04415761])

3:


In [115]:
test_mat = np.identity(11)[1:]
f_test = result.f_test(test_mat)
print(f_test)

<F test: F=1.9787545710296293, p=0.044958975270456, df_denom=89, df_num=10>


In [80]:
def select_and_check_2(interval, index):
    return (interval[0] < coff[index] < interval[1])

In [110]:
def select_and_check_vector_2(results):
    intervals = results.conf_int()
    test_mat = np.identity(11)[1:]
    pval = results.f_test(test_mat).pvalue
    return np.array([select_and_check_2(intervals[i], i) for i in range(11)]) if pval < 0.1 else np.full(11, np.NAN)

In [111]:
array2 = np.array([select_and_check_vector_2(gen_results(100)) for i in range(5000)])

In [129]:
array2[~np.isnan(array2).any(axis=1)].mean(axis=0)

array([0.92198582, 0.82978723, 0.85106383, 0.85390071, 0.85531915,
       0.85673759, 0.85531915, 0.84822695, 0.85957447, 0.85106383,
       0.86524823])